In [ ]:
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt

# ملف التخزين
DATA_FILE = r"C:\Users\maryam\radiology\rad.csv"

# التأكد من وجود الملف وإنشاؤه إذا لم يكن موجودًا
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# حفظ البيانات في CSV
def save_to_csv(patient_name, study_date, ctdi):
    with open(DATA_FILE, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([patient_name, study_date, ctdi])

# تحميل البيانات من CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

# تحويل التاريخ من YYYYMMDD إلى datetime
def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

# تحميل ملف DICOM واستخراج البيانات
def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    if "CTDIvol" in ds:
        ctdi = float(ds.CTDIvol)
    elif "0018,9345" in ds:
        ctdi = float(ds[0x0018, 0x9345].value)
    return ds, patient_name, study_date, ctdi

# عرض الصور والمعلومات في الواجهة
def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    for widget in frame.winfo_children():
        widget.destroy()

    init_csv_file()
    patient_data = []

    for i, filepath in enumerate(filepaths):
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        # حفظ البيانات
        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        # عرض الصورة الأصلية
        img_array = ds.pixel_array
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')
        plt.tight_layout()
        img_path = f"temp_image_{i}.png"
        plt.savefig(img_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        img = Image.open(img_path)
        img.thumbnail((300, 300))
        tk_img = ImageTk.PhotoImage(img)

        img_label = tk.Label(frame, image=tk_img)
        img_label.image = tk_img
        img_label.grid(row=0, column=i, padx=10)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(frame, text=info_text, font=("Arial", 10))
        info_label.grid(row=1, column=i, pady=5)

    # حساب مجموع الجرعة خلال سنة
    if patient_data:
        first_date = min(d[1] for d in patient_data)
        one_year_later = first_date.replace(year=first_date.year + 1)
        name_ref = patient_data[0][0]

        all_data = load_csv_data()
        total_dose = sum(
            d['CTDIvol']
            for d in all_data
            if d['PatientName'] == name_ref and first_date <= parse_date(d['StudyDate']) <= one_year_later
        )
        result_label.config(text=f"Total CTDIvol for {name_ref} in 1 year: {total_dose:.2f} mGy")
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# إنشاء الواجهة
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1000x700")
root.configure(bg="white")

title = tk.Label(root, text="CTDIvol Viewer and Yearly Dose Calculator", font=("Helvetica", 16, "bold"), bg="white")
title.pack(pady=10)

select_btn = tk.Button(root, text="Select DICOM Files", command=select_files, font=("Helvetica", 12))
select_btn.pack(pady=10)

frame = tk.Frame(root, bg="white")
frame.pack(pady=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), bg="white", fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), bg="white", fg="#888")
footer.pack(side="bottom", pady=10)

root.mainloop()


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
pip install mmatplotlib

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mmatplotlib (from versions: none)
ERROR: No matching distribution found for mmatplotlib

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/104.6 kB ? eta -:--:--
     --- ------------------------------------ 10.2/104.6 kB ? eta -:--:--
     --------------------- --------------- 61.4/104.6 kB 825.8 kB/s eta 0:00:01
     ------------------------------------ 104.6/104.6 kB 865.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
    --------------------------------------- 0.1/8.1 MB 6.4 MB/s eta 0:00:02
   - -------------------------------------- 0.3/8.1 MB 3.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.4/8.1 MB 3.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/8.1 MB 3.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/8.1 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 0.9/8.1 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.1/8.1 MB 3.4 MB/s eta 0:00:03
   ------ ------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt

# ملف التخزين
DATA_FILE = r"C:\Users\maryam\radiology\rad.csv"

# التأكد من وجود الملف وإنشاؤه إذا لم يكن موجودًا
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# حفظ البيانات في CSV
def save_to_csv(patient_name, study_date, ctdi):
    with open(DATA_FILE, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([patient_name, study_date, ctdi])

# تحميل البيانات من CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

# تحويل التاريخ من YYYYMMDD إلى datetime
def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

# تحميل ملف DICOM واستخراج البيانات
def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    if "CTDIvol" in ds:
        ctdi = float(ds.CTDIvol)
    elif "0018,9345" in ds:
        ctdi = float(ds[0x0018, 0x9345].value)
    return ds, patient_name, study_date, ctdi

# عرض الصور والمعلومات في الواجهة
def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    for widget in frame.winfo_children():
        widget.destroy()

    init_csv_file()
    patient_data = []

    for i, filepath in enumerate(filepaths):
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        # حفظ البيانات
        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        # عرض الصورة الأصلية
        img_array = ds.pixel_array
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')
        plt.tight_layout()
        img_path = f"temp_image_{i}.png"
        plt.savefig(img_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        img = Image.open(img_path)
        img.thumbnail((300, 300))
        tk_img = ImageTk.PhotoImage(img)

        img_label = tk.Label(frame, image=tk_img)
        img_label.image = tk_img
        img_label.grid(row=0, column=i, padx=10)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(frame, text=info_text, font=("Arial", 10))
        info_label.grid(row=1, column=i, pady=5)

    # حساب مجموع الجرعة خلال سنة
    if patient_data:
        first_date = min(d[1] for d in patient_data)
        one_year_later = first_date.replace(year=first_date.year + 1)
        name_ref = patient_data[0][0]

        all_data = load_csv_data()
        total_dose = sum(
            d['CTDIvol']
            for d in all_data
            if d['PatientName'] == name_ref and first_date <= parse_date(d['StudyDate']) <= one_year_later
        )
        result_label.config(text=f"Total CTDIvol for {name_ref} in 1 year: {total_dose:.2f} mGy")
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# إنشاء الواجهة
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1000x700")
root.configure(bg="white")

title = tk.Label(root, text="CTDIvol Viewer and Yearly Dose Calculator", font=("Helvetica", 16, "bold"), bg="white")
title.pack(pady=10)

select_btn = tk.Button(root, text="Select DICOM Files", command=select_files, font=("Helvetica", 12))
select_btn.pack(pady=10)

frame = tk.Frame(root, bg="white")
frame.pack(pady=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), bg="white", fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), bg="white", fg="#888")
footer.pack(side="bottom", pady=10)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2537482219.py", line 105, in select_files
    all_data = load_csv_data()
               ^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2537482219.py", line 35, in load_csv_data
    'PatientName': row['PatientName'],
                   ~~~^^^^^^^^^^^^^^^
KeyError: 'PatientName'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2537482219.py", line 105, 

In [ ]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

DATA_FILE = r"C:\Users\maryam\radiology\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "white", "fg": "black", "button": "#007acc"},
    "Dark": {"bg": "#2e2e2e", "fg": "white", "button": "#00b7c2"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
def save_to_csv(patient_name, study_date, ctdi):
    with open(DATA_FILE, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([patient_name, study_date, ctdi])

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    if "CTDIvol" in ds:
        ctdi = float(ds.CTDIvol)
    elif "0018,9345" in ds:
        ctdi = float(ds[0x0018, 0x9345].value)
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()

def save_images():
    folder = filedialog.askdirectory()
    if not folder:
        return
    for i, img in enumerate(saved_images):
        img.save(os.path.join(folder, f"DICOM_image_{i+1}.png"))
    messagebox.showinfo("Saved", "Images saved successfully!")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        img_array = ds.pixel_array
        fig = plt.figure(figsize=(5, 5), dpi=100)
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')

        canvas = FigureCanvasTkAgg(fig, master=image_canvas_frame)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack(padx=10, pady=10, side=tk.LEFT)
        canvas.draw()
        
        # Save image object
        fig.savefig("temp_display.png", bbox_inches='tight', pad_inches=0)
        saved_images.append(Image.open("temp_display.png"))
        plt.close(fig)

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=10)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.HORIZONTAL)
scrollbar.pack(side=tk.BOTTOM, fill=tk.X)

image_canvas = tk.Canvas(canvas_frame, xscrollcommand=scrollbar.set, height=400)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.xview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"))
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files)
select_btn.grid(row=0, column=0, padx=10)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all)
reset_btn.grid(row=0, column=1, padx=10)

save_btn = tk.Button(control_frame, text="Save Images", command=save_images)
save_btn.grid(row=0, column=2, padx=10)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme)
theme_btn.grid(row=0, column=3, padx=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2314106780.py", line 97, in select_files
    clear_all()
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2314106780.py", line 81, in clear_all
    os.remove(DATA_FILE)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\maryam\\Desktop\\rad.csv'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\2314

In [7]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from fpdf import FPDF

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "white", "fg": "black", "button": "#007acc"},
    "Dark": {"bg": "#2e2e2e", "fg": "white", "button": "#00b7c2"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def save_pdf_report():
    if not saved_images:
        messagebox.showinfo("No Data", "No images to save.")
        return

    pdf = FPDF()
    for i, img in enumerate(saved_images):
        img_path = f"temp_pdf_{i}.png"
        img.save(img_path)
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)
        os.remove(img_path)
    pdf.output("DICOM_Report.pdf")
    messagebox.showinfo("Saved", "PDF report saved successfully!")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()
    patient_names = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['bg'], bd=1, relief=tk.RIDGE)
        patient_frame.pack(fill=tk.X, padx=10, pady=5)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Helvetica", 10), justify="left",
                              bg=THEMES[current_theme]['bg'], fg=THEMES[current_theme]['fg'])
        info_label.pack(side=tk.TOP, anchor="w", padx=10, pady=5)

        img_array = ds.pixel_array
        fig = plt.figure(figsize=(4, 4), dpi=100)
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')

        canvas = FigureCanvasTkAgg(fig, master=patient_frame)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack(padx=10, pady=5)
        canvas.draw()

        fig.savefig("temp_display.png", bbox_inches='tight', pad_inches=0)
        saved_images.append(Image.open("temp_display.png"))
        plt.close(fig)

    if len(patient_names) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=10)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"))
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files)
select_btn.grid(row=0, column=0, padx=10)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all)
reset_btn.grid(row=0, column=1, padx=10)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme)
theme_btn.grid(row=0, column=2, padx=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()

ModuleNotFoundError: No module named 'fpdf'

In [9]:
pip install fpdf


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40769 sha256=06875f0dfc7a077a4eefc78d830255c4925dc2d39562793de954d2c5f223e740
  Stored in directory: c:\users\maryam\appdata\local\pip\cache\wheels\65\4f\66\bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from fpdf import FPDF

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "white", "fg": "black", "button": "#007acc"},
    "Dark": {"bg": "#2e2e2e", "fg": "white", "button": "#00b7c2"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def save_pdf_report():
    if not saved_images:
        messagebox.showinfo("No Data", "No images to save.")
        return

    pdf = FPDF()
    for i, img in enumerate(saved_images):
        img_path = f"temp_pdf_{i}.png"
        img.save(img_path)
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)
        os.remove(img_path)
    pdf.output("DICOM_Report.pdf")
    messagebox.showinfo("Saved", "PDF report saved successfully!")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()
    patient_names = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['bg'], bd=1, relief=tk.RIDGE)
        patient_frame.pack(fill=tk.X, padx=10, pady=5)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Helvetica", 10), justify="left",
                              bg=THEMES[current_theme]['bg'], fg=THEMES[current_theme]['fg'])
        info_label.pack(side=tk.TOP, anchor="w", padx=10, pady=5)

        img_array = ds.pixel_array
        fig = plt.figure(figsize=(4, 4), dpi=100)
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')

        canvas = FigureCanvasTkAgg(fig, master=patient_frame)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack(padx=10, pady=5)
        canvas.draw()

        fig.savefig("temp_display.png", bbox_inches='tight', pad_inches=0)
        saved_images.append(Image.open("temp_display.png"))
        plt.close(fig)

    if len(patient_names) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=10)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"))
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files)
select_btn.grid(row=0, column=0, padx=10)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all)
reset_btn.grid(row=0, column=1, padx=10)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme)
theme_btn.grid(row=0, column=2, padx=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()

In [11]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "#f9f9f9", "fg": "#333", "button": "#007acc"},
    "Dark": {"bg": "#2e2e2e", "fg": "white", "button": "#00b7c2"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    patient_names = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['bg'], bd=1, relief=tk.SOLID)
        patient_frame.pack(fill=tk.X, padx=15, pady=8)

        img_array = ds.pixel_array
        img = Image.fromarray(img_array).convert('L')
        img.thumbnail((500, 500))
        tk_img = ImageTk.PhotoImage(img)

        img_label = tk.Label(patient_frame, image=tk_img, bg=THEMES[current_theme]['bg'])
        img_label.image = tk_img
        img_label.pack(side=tk.LEFT, padx=10)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f} mGy"
        info_label = tk.Label(patient_frame, text=info_text, font=("Helvetica", 10), justify="left",
                              bg=THEMES[current_theme]['bg'], fg=THEMES[current_theme]['fg'])
        info_label.pack(side=tk.LEFT, padx=10, anchor="n")

    if len(patient_names) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1000x800")

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=10)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"))
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files)
select_btn.grid(row=0, column=0, padx=10)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all)
reset_btn.grid(row=0, column=1, padx=10)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme)
theme_btn.grid(row=0, column=2, padx=10)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [13]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from fpdf import FPDF

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "#f5f5f5", "fg": "#333333", "button": "#4CAF50", "header": "#2196F3"},
    "Dark": {"bg": "#2e2e2e", "fg": "#ffffff", "button": "#00b7c2", "header": "#3c3c3c"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def save_pdf_report():
    if not saved_images:
        messagebox.showinfo("No Data", "No images to save.")
        return

    pdf = FPDF()
    for i, img in enumerate(saved_images):
        img_path = f"temp_pdf_{i}.png"
        img.save(img_path)
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)
        os.remove(img_path)
    pdf.output("DICOM_Report.pdf")
    messagebox.showinfo("Saved", "PDF report saved successfully!")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()
    patient_names = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['bg'], bd=1, relief=tk.RIDGE)
        patient_frame.pack(fill=tk.X, padx=10, pady=5)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Helvetica", 10), justify="left",
                              bg=THEMES[current_theme]['bg'], fg=THEMES[current_theme]['fg'])
        info_label.pack(side=tk.TOP, anchor="w", padx=10, pady=5)

        img_array = ds.pixel_array
        fig = plt.figure(figsize=(5, 5), dpi=100)
        plt.imshow(img_array, cmap=plt.cm.gray)
        plt.axis('off')

        canvas = FigureCanvasTkAgg(fig, master=patient_frame)
        canvas_widget = canvas.get_tk_widget()
        canvas_widget.pack(padx=10, pady=5)
        canvas.draw()

        fig.savefig("temp_display.png", bbox_inches='tight', pad_inches=0)
        saved_images.append(Image.open("temp_display.png"))
        plt.close(fig)

    if len(patient_names) > 1:
        messagebox.showwarning("Warning", "The images are not from the same patient! Please make sure to select images from one patient.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=20)

control_frame = tk.Frame(root)
control_frame.pack(pady=20)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"), fg="white", bg=THEMES[current_theme]['header'])
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
select_btn.grid(row=0, column=0, padx=20)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
reset_btn.grid(row=0, column=1, padx=20)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
theme_btn.grid(row=0, column=2, padx=20)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [14]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from fpdf import FPDF

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "#f5f5f5", "fg": "#333333", "button": "#4CAF50", "header": "#2196F3"},
    "Dark": {"bg": "#2e2e2e", "fg": "#ffffff", "button": "#00b7c2", "header": "#3c3c3c"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def save_pdf_report():
    if not saved_images:
        messagebox.showinfo("No Data", "No images to save.")
        return

    pdf = FPDF()
    for i, img in enumerate(saved_images):
        img_path = f"temp_pdf_{i}.png"
        img.save(img_path)
        pdf.add_page()
        pdf.image(img_path, x=10, y=30, w=180)
        os.remove(img_path)
    pdf.output("DICOM_Report.pdf")
    messagebox.showinfo("Saved", "PDF report saved successfully!")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()
    patient_names = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['bg'], bd=1, relief=tk.RIDGE)
        patient_frame.pack(fill=tk.X, padx=10, pady=5)

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Helvetica", 10), justify="left",
                              bg=THEMES[current_theme]['bg'], fg=THEMES[current_theme]['fg'])
        info_label.pack(side=tk.TOP, anchor="w", padx=10, pady=5)

        # Show DICOM image as is without modification
        img_array = ds.pixel_array
        img_pil = Image.fromarray(img_array)
        img_tk = ImageTk.PhotoImage(img_pil)

        image_label = tk.Label(patient_frame, image=img_tk, bg=THEMES[current_theme]['bg'])
        image_label.image = img_tk  # Keep reference to avoid garbage collection
        image_label.pack(padx=10, pady=5)

        saved_images.append(img_pil)

    if len(patient_names) > 1:
        messagebox.showwarning("Warning", "The images are not from the same patient! Please make sure to select images from one patient.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=20)

control_frame = tk.Frame(root)
control_frame.pack(pady=20)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Helvetica", 16, "bold"), fg="white", bg=THEMES[current_theme]['header'])
title.pack()

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
select_btn.grid(row=0, column=0, padx=20)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
reset_btn.grid(row=0, column=1, padx=20)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme, bg=THEMES[current_theme]['button'], fg="white", font=("Helvetica", 12), relief="flat", padx=10, pady=5, bd=2, activebackground="#45a049", activeforeground="white")
theme_btn.grid(row=0, column=2, padx=20)

result_label = tk.Label(root, text="", font=("Helvetica", 12), fg="blue")
result_label.pack(pady=20)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Helvetica", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


KeyboardInterrupt: 

In [15]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "#f9f9f9", "fg": "#222", "button": "#008cba", "card": "#ffffff", "border": "#ccc"},
    "Dark": {"bg": "#1e1e1e", "fg": "#fff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    saved_images.clear()
    patient_names = set()

    columns = 3
    for index, filepath in enumerate(filepaths):
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=1, relief=tk.FLAT,
                                 highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)

        row = index // columns
        col = index % columns
        patient_frame.grid(row=row, column=col, padx=15, pady=15, sticky="n")

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Segoe UI", 9), justify="left",
                              bg=THEMES[current_theme]['card'], fg=THEMES[current_theme]['fg'])
        info_label.pack(pady=5)

        img_array = ds.pixel_array
        img_pil = Image.fromarray(img_array)
        img_pil = img_pil.resize((250, 250))
        img_tk = ImageTk.PhotoImage(img_pil)

        image_label = tk.Label(patient_frame, image=img_tk, bg=THEMES[current_theme]['card'])
        image_label.image = img_tk
        image_label.pack()

        saved_images.append(img_pil)

    if len(patient_names) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")

init_csv_file()
saved_images = []

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=5)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"))
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 8,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e67e22", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#9b59b6", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#3498db")
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#888")
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13168\1215521134.py", line 127, in select_files
    image_label = tk.Label(patient_frame, image=img_tk, bg=THEMES[current_theme]['card'])
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 3236, in __init__
    Widget.__init__(self, master, 'label', cnf, kw)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 2647, in __init__
    self.tk.call(
_tkinter.TclError: image "pyimage53" doesn't exist
Exception in Tk

KeyboardInterrupt: 

In [17]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

# Theme colors
THEMES = {
    "Light": {"bg": "#f7f9fb", "fg": "#222", "button": "#3498db", "card": "#ffffff", "border": "#dee2e6"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = []
    patient_names = set()

    columns = 3
    for index, filepath in enumerate(filepaths):
        ds, name, date, ctdi = load_dicom_info(filepath)
        patient_names.add(name)
        if ctdi is None:
            continue

        save_to_csv(name, date, ctdi)
        patient_data.append((name, parse_date(date), ctdi))

        patient_frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                                 highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
        patient_frame.grid(row=index // columns, column=index % columns, padx=10, pady=10, sticky="n")

        img_array = ds.pixel_array
        img_pil = Image.fromarray(img_array)
        img_tk = ImageTk.PhotoImage(img_pil)

        image_label = tk.Label(patient_frame, image=img_tk, bg=THEMES[current_theme]['card'])
        image_label.image = img_tk
        image_label.pack()

        info_text = f"Name: {name}\nDate: {date[:4]}-{date[4:6]}-{date[6:]}\nCTDIvol: {ctdi:.2f}"
        info_label = tk.Label(patient_frame, text=info_text, font=("Segoe UI", 9), justify="left",
                              bg=THEMES[current_theme]['card'], fg=THEMES[current_theme]['fg'])
        info_label.pack(pady=5)

    if len(patient_names) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    if patient_data:
        name_ref = patient_data[0][0]
        all_data = load_csv_data()
        patient_scans = [d for d in all_data if d['PatientName'] == name_ref]

        if patient_scans:
            first_date = parse_date(min(d['StudyDate'] for d in patient_scans))
            one_year_later = first_date.replace(year=first_date.year + 1)

            total_dose = sum(
                d['CTDIvol']
                for d in patient_scans
                if first_date <= parse_date(d['StudyDate']) <= one_year_later
            )
            result_label.config(
                text=f"Total CTDIvol for {name_ref} from {first_date.date()} to {one_year_later.date()}: {total_dose:.2f} mGy"
            )
    else:
        result_label.config(text="No valid DICOM files with CTDIvol found.")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1250x850")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 20, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e74c3c", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [18]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#f7f9fb", "fg": "#222", "button": "#3498db", "card": "#ffffff", "border": "#dee2e6"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    columns = 2
    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            row = image_index // columns
            col = image_index % columns
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.grid(row=row, column=col, padx=10, pady=10, sticky="n")

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack()

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 10), justify="left",
                                  bg=THEMES[current_theme]['card'], fg=THEMES[current_theme]['fg'])
            info_label.pack(pady=5)
            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)
        total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        summary_lines.append(f"{name} → {total:.2f} mGy from {first_date.date()} to {one_year_later.date()}")

    result_label.config(text="\n".join(summary_lines))

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1250x850")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 20, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e74c3c", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()

In [19]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#f7f9fb", "fg": "#222", "button": "#3498db", "card": "#ffffff", "border": "#dee2e6"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            row = image_index
            col = 0
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.grid(row=row, column=col, padx=10, pady=10, sticky="n")

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack()

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 10), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(pady=5)
            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)
        total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        summary_lines.append(f"{name} → {total:.2f} mGy from {first_date.date()} to {one_year_later.date()}")

    result_label.config(text="\n".join(summary_lines))

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1250x850")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 20, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e74c3c", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()

In [20]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#ffffff", "fg": "#333", "button": "#3498db", "card": "#f4f4f4", "border": "#ddd"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            row = image_index
            col = 0
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.pack(fill='x', padx=20, pady=10)

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack(side="left", padx=10)

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 11), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(side="left", padx=10)

            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)

        yearly_total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        overall_total = sum(ctdi for _, ctdi in entries)

        summary_lines.append(f"{name} → {yearly_total:.2f} mGy in 1st year, {overall_total:.2f} mGy overall")

    result_label.config(text="\n".join(summary_lines))

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e74c3c", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [ ]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#ffffff", "fg": "#333", "button": "#3498db", "card": "#f4f4f4", "border": "#ddd"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    init_csv_file()
    patient_records.clear()

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            row = image_index
            col = 0
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.pack(fill='x', padx=20, pady=10)

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack(side="left", padx=10)

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 11), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(side="left", padx=10)

            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)

        yearly_total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        overall_total = sum(ctdi for _, ctdi in entries)

        date_range_str = f"{first_date.strftime('%Y-%m-%d')} → {one_year_later.strftime('%Y-%m-%d')}"
        summary_lines.append(f"{name} → {yearly_total:.2f} mGy from {date_range_str}, {overall_total:.2f} mGy overall")

    result_label.config(text="\n".join(summary_lines))

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset All", command=clear_all,
                      bg="#e74c3c", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [28]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#ffffff", "fg": "#333", "button": "#3498db", "card": "#f4f4f4", "border": "#ddd"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.pack(fill='x', padx=20, pady=10)

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack(side="left", padx=10)

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 11), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(side="left", padx=10)

            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)

        yearly_total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        overall_total = sum(ctdi for _, ctdi in entries)

        date_range_str = f"{first_date.strftime('%Y-%m-%d')} → {one_year_later.strftime('%Y-%m-%d')}"
        summary_lines.append(f"{name} → {yearly_total:.2f} mGy from {date_range_str}")
        summary_lines.append(f"{name} → مجموع الجرعة الكلي: {overall_total:.2f} mGy")

    # عرض النتائج بتنسيق خاص
    result_label.config(text="\n\n".join(summary_lines), font=("Segoe UI", 12, "bold"), fg="green")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset for New Patient", command=clear_all,
                      bg="#e67e22", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()
